# Question 8: 線形計画法 (5)

=============================================================================================

問題）変数 x と y が次の条件を満たすとき、  
(1) x+y が最大となる x と y のそれぞれ浮動小数点と整数の組み合わせを求めよ。  
(2) x+y が最小となる x と y の正の整数の組み合わせを求めよ。  

＜条件＞  
4 x + y <= 12  
0.5 x + y <=4  

数理最適化のライブラリー Google OR-Tools を用いて解きなさい。

=============================================================================================

(1) Google OR-Tools を用いて、浮動小数点での目的関数 (x + y) が最大になるようにといてみましょう。

In [1]:
!pip install ortools

     |████████████████████████████████| 14.6 MB 4.8 MB/s 
     |████████████████████████████████| 1.1 MB 58.9 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3


In [2]:
from ortools.linear_solver import pywraplp

In [42]:
# ソルバーの定義
solver = pywraplp.Solver.CreateSolver('GLOP')

# 変数の定義
infinity = solver.infinity()
x = solver.NumVar(0.0, infinity, 'x')
y = solver.NumVar(0.0, infinity, 'y')

# 目的関数と問題の定義
solver.Maximize(x + y)

# 制約条件の定義
c0 = solver.Add(-4*x + 12 >= y, 'c0')
c1 = solver.Add(-1/2*x+4 >= y, 'c1')

In [43]:
# 問題を解いて結果を表示する関数
def SolveAndPrint(solver, variable_list, constraint_list):
    """Solve the problem and print the solution."""
    print('Number of variables = %d' % solver.NumVariables())
    print('Number of constraints = %d' % solver.NumConstraints())

    result_status = solver.Solve()

    # The problem has an optimal solution.
    assert result_status == pywraplp.Solver.OPTIMAL

    # The solution looks legit (when using solvers others than
    # GLOP_LINEAR_PROGRAMMING, verifying the solution is highly recommended!).
    assert solver.VerifySolution(1e-7, True)

    print('Problem solved in %f milliseconds' % solver.wall_time())

    # The objective value of the solution.
    print('Optimal objective value = %f' % solver.Objective().Value())

    # The value of each variable in the solution.
    for variable in variable_list:
        print('%s = %f' % (variable.name(), variable.solution_value()))

    print('Advanced usage:')
    print('Problem solved in %d iterations' % solver.iterations())
    for variable in variable_list:
        print('%s: reduced cost = %f' %
              (variable.name(), variable.reduced_cost()))
    activities = solver.ComputeConstraintActivities()
    for i, constraint in enumerate(constraint_list):
        print(('constraint %d: dual value = %f\n'
               '               activity = %f' %
               (i, constraint.dual_value(), activities[constraint.index()])))

In [44]:
%%time
# 解く
SolveAndPrint(solver, [x, y], [c0, c1])

Number of variables = 2
Number of constraints = 2
Problem solved in 5710.000000 milliseconds
Optimal objective value = 5.142857
x = 2.285714
y = 2.857143
Advanced usage:
Problem solved in 2 iterations
x: reduced cost = 0.000000
y: reduced cost = 0.000000
constraint 0: dual value = -0.142857
               activity = -12.000000
constraint 1: dual value = -0.857143
               activity = -4.000000
CPU times: user 5.07 ms, sys: 1.03 ms, total: 6.1 ms
Wall time: 7.13 ms


In [45]:
# The objective value of the solution.
print('Optimal objective value = %f' % solver.Objective().Value())

# The value of each variable in the solution.
for variable in  [x, y]:
    print('%s = %f' % (variable.name(), variable.solution_value()))

Optimal objective value = 5.142857
x = 2.285714
y = 2.857143


Google OR-Tools の線形計画では、「GLOP」「GLPK_LP」「CLP」の３種類のソルバーが使えるようです。それぞれの比較をしてみましょう。

In [49]:
import time

def RunLiniarProblem(optimization_problem_type):
  start = time.time()
  # ソルバーの定義
  solver = pywraplp.Solver.CreateSolver(optimization_problem_type)
  if not solver:
    print(optimization_problem_type + " is not applicable for this problem")
    return

  # 変数の定義
  # x and y are continuous non-negative variables.
  infinity = solver.infinity()
  x = solver.NumVar(0.0, infinity, 'x')
  y = solver.NumVar(0.0, infinity, 'y')

  # 目的関数と問題の定義
  solver.Maximize(x + y)

  # 制約条件の定義
  c0 = solver.Add(-4*x + 12 >= y, 'c0')
  c1 = solver.Add(-1/2*x+4 >= y, 'c1')

  # 解く
  result_status = solver.Solve()
  # The problem has an optimal solution.
  assert result_status == pywraplp.Solver.OPTIMAL

  # optimization_problem_type
  print("Optimizatio problem type:", optimization_problem_type)

  # The objective value of the solution.
  print('Optimal objective value = %f' % solver.Objective().Value())

  # The value of each variable in the solution.
  for variable in  [x, y]:
      print('%s = %f' % (variable.name(), variable.solution_value()))
  
  # 計算時間
  print('Solving time:', time.time() - start, 's')

In [52]:
RunLiniarProblem('GLOP')
print('\n')
RunLiniarProblem('GLPK_LP')
print('\n')
RunLiniarProblem('CLP')

Optimizatio problem type: GLOP
Optimal objective value = 5.142857
x = 2.285714
y = 2.857143
Solving time: 0.004686117172241211 s


GLPK_LP is not applicable for this problem


Optimizatio problem type: CLP
Optimal objective value = 5.142857
x = 2.285714
y = 2.857143
Solving time: 0.0008823871612548828 s


GLPK_LP は対応していなかったようです。

次に整数解を解きます。整数問題の場合はソルバーに「SCIP」を用います。

In [59]:
def RunIntLiniarProblem(optimization_problem_type):
  start = time.time()
  # ソルバーの定義
  solver = pywraplp.Solver.CreateSolver(optimization_problem_type)
  if not solver:
    print(optimization_problem_type + " is not applicable for this problem")
    return

  # 変数の定義
  # x and y are integer non-negative variables.
  infinity = solver.infinity()
  x = solver.IntVar(0.0, infinity, 'x') # 整数として定義
  y = solver.IntVar(0.0, infinity, 'y') # 整数として定義

  # 目的関数と問題の定義
  solver.Maximize(x + y)

  # 制約条件の定義
  c0 = solver.Add(-4*x + 12 >= y, 'c0')
  c1 = solver.Add(-1/2*x+4 >= y, 'c1')

  # 解く
  result_status = solver.Solve()
  # The problem has an optimal solution.
  assert result_status == pywraplp.Solver.OPTIMAL

  # optimization_problem_type
  print("Optimizatio problem type:", optimization_problem_type)

  # The objective value of the solution.
  print('Optimal objective value = %f' % solver.Objective().Value())

  # The value of each variable in the solution.
  for variable in  [x, y]:
      print('%s = %f' % (variable.name(), variable.solution_value()))
  
  # 計算時間
  print('Solving time:', time.time() - start, 's')

In [60]:
RunIntLiniarProblem('SCIP')

Optimizatio problem type: SCIP
Optimal objective value = 5.000000
x = 2.000000
y = 3.000000
Solving time: 0.00842428207397461 s


(2) 次に最小問題を解きます。整数解のみ求めましょう。

In [68]:
def RunIntLiniarProblem(optimization_problem_type):
  start = time.time()
  # ソルバーの定義
  solver = pywraplp.Solver.CreateSolver(optimization_problem_type)
  if not solver:
    print(optimization_problem_type + " is not applicable for this problem")
    return

  # 変数の定義
  # x and y are integer non-negative variables.
  infinity = solver.infinity()
  x = solver.IntVar(1.0, infinity, 'x') # 正の整数として定義（0は含まない）
  y = solver.IntVar(1.0, infinity, 'y') # 正の整数として定義（0は含まない）

  # 目的関数と問題の定義
  solver.Minimize(x + y)

  # 制約条件の定義
  c0 = solver.Add(-4*x + 12 >= y, 'c0')
  c1 = solver.Add(-1/2*x+4 >= y, 'c1')

  # 解く
  result_status = solver.Solve()
  # The problem has an optimal solution.
  assert result_status == pywraplp.Solver.OPTIMAL

  # optimization_problem_type
  print("Optimizatio problem type:", optimization_problem_type)

  # The objective value of the solution.
  print('Optimal objective value = %f' % solver.Objective().Value())

  # The value of each variable in the solution.
  for variable in  [x, y]:
      print('%s = %f' % (variable.name(), variable.solution_value()))
  
  # 計算時間
  print('Solving time:', time.time() - start, 's')

In [69]:
RunIntLiniarProblem('SCIP')

Optimizatio problem type: SCIP
Optimal objective value = 2.000000
x = 1.000000
y = 1.000000
Solving time: 0.002900838851928711 s
